In [1]:
# Necessary Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_auc_score
import optuna
import logging

optuna.logging.set_verbosity(optuna.logging.WARNING)



In [2]:
# Exploratory Data Analysis
# Load Data
women_train = pd.read_csv('/kaggle/input/widsdatathon2024-challenge1/training.csv')
women_test = pd.read_csv("/kaggle/input/widsdatathon2024-challenge1/test.csv")
sample_submission = pd.read_csv("/kaggle/input/widsdatathon2024-challenge1/sample_submission.csv")


In [3]:
# Feature Engineering
# Drop unnecessary columns for modeling
drop_columns = ['metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'bmi','patient_race']
women_train = women_train.drop(columns=drop_columns)
women_test = women_test.drop(columns=drop_columns)



In [4]:
# Engineering new features
# Calculate age from birth year
women_train['age'] = 2024 - women_train['patient_age']
women_test['age'] = 2024 - women_test['patient_age']

# Normalize ozone, PM25, and NO2 by population density
women_train['ozone_normalized'] = women_train['Ozone'] / women_train['density']
women_train['pm25_normalized'] = women_train['PM25'] / women_train['density']
women_train['no2_normalized'] = women_train['N02'] / women_train['density']

women_test['ozone_normalized'] = women_test['Ozone'] / women_test['density']
women_test['pm25_normalized'] = women_test['PM25'] / women_test['density']
women_test['no2_normalized'] = women_test['N02'] / women_test['density']

In [5]:
# Text preprocessing for 'breast_cancer_diagnosis_desc'
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation and special characters
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    return text

women_train['breast_cancer_diagnosis_desc'] = women_train['breast_cancer_diagnosis_desc'].apply(preprocess_text)
women_test['breast_cancer_diagnosis_desc'] = women_test['breast_cancer_diagnosis_desc'].apply(preprocess_text)


In [6]:
# Define features and target variable
X = women_train.drop(columns=['DiagPeriodL90D'])
y = women_train['DiagPeriodL90D']

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Preprocessing for numerical and categorical data
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

text_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer())
])

# Bundle preprocessing for numerical, categorical, and text data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
        ('text', text_transformer, 'breast_cancer_diagnosis_desc')
    ])

In [8]:
# Model Building and Evaluation
# Define XGBoost model with Optuna hyperparameter optimization
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'random_state': 42
    }

    xgb_model = XGBClassifier(**params)

    clf = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', xgb_model)])
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, y_pred)

    return auc


In [9]:
# Define CatBoost model with Optuna hyperparameter optimization
def objective_catboost(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 200),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0),
        'random_state': 42,
        'verbose': 0
    }

    catboost_model = CatBoostClassifier(**params)

    clf = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', catboost_model)])
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, y_pred)

    return auc

In [10]:
# Create and run the Optuna study for XGBoost
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=50)

# Get the best parameters for XGBoost
best_params_xgb = study_xgb.best_params


In [11]:
# Train XGBoost model with best hyperparameters
xgb_model = XGBClassifier(**best_params_xgb, random_state=42)
clf_xgb = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', xgb_model)])
clf_xgb.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['patient_id', 'patient_zip3', 'patient_age', 'population', 'density',
       'age_median', 'age_under_10', 'age_10_to_19', 'age_20s', 'age_30s',
       'age_40s', 'age_50s', 'age_60s', 'age_70s', 'age_over_80', 'male',
       'female',...
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.01712543711090616, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=6,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=272,
                               n_jobs=None, num_parallel_tree=None,
                               random_state=42, ...))])

In [12]:
# Evaluate XGBoost model
y_pred_xgb = clf_xgb.predict_proba(X_valid)[:, 1]
auc_xgb = roc_auc_score(y_valid, y_pred_xgb)
print(f'XGBoost: AUC = {auc_xgb:.4f}')


XGBoost: AUC = 0.8015


In [13]:
# Create and run the Optuna study for CatBoost
study_catboost = optuna.create_study(direction='maximize')
study_catboost.optimize(objective_catboost, n_trials=50)

# Get the best parameters for CatBoost
best_params_catboost = study_catboost.best_params

In [14]:
# Train CatBoost model with best hyperparameters
catboost_model = CatBoostClassifier(**best_params_catboost, random_state=42, verbose=0)
clf_catboost = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', catboost_model)])
clf_catboost.fit(X_train, y_train)



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['patient_id', 'patient_zip3', 'patient_age', 'population', 'density',
       'age_median', 'age_under_10', 'age_10_to_19', 'age_20s', 'age_30s',
       'age_40s', 'age_50s', 'age_60s', 'age_70s', 'age_over_80', 'male',
       'female',...
                                                  Index(['payer_type', 'patient_state', 'patient_gender',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'Region', 'Division'],
      dtype='object')),
                                                 ('text',
                                                  Pipeline(steps=[('tfidf',
                                                                   TfidfVectorizer())]),
                                                  'breast_cancer_diagnosis_desc')])),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x7ef0d36f8220>)])

In [15]:
# Evaluate CatBoost model
y_pred_catboost = clf_catboost.predict_proba(X_valid)[:, 1]
auc_catboost = roc_auc_score(y_valid, y_pred_catboost)
print(f'CatBoost: AUC = {auc_catboost:.4f}')



CatBoost: AUC = 0.8026


In [16]:
# Ensemble XGBoost and CatBoost models
voting_clf = VotingClassifier([('xgb', clf_xgb), ('catboost', clf_catboost)], voting='soft')
voting_clf.fit(X_train, y_train)


VotingClassifier(estimators=[('xgb',
                              Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(transformers=[('num',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer()),
                                                                                                ('scaler',
                                                                                                 StandardScaler())]),
                                                                                Index(['patient_id', 'patient_zip3', 'patient_age', 'population', 'density',
       'age_median', 'age_under_10', 'age_10_to_19', 'age_20s', 'age_30s',
       'age_40s', 'age_50s', 'age_60s', 'age...
                                                                                Index(['payer_type', 'patient_state', 'patient_gender',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'Region', 'Division'],
      dtype='object')),
                                                                               ('text',
                                                                                Pipeline(steps=[('tfidf',
                                                                                                 TfidfVectorizer())]),
                                                                                'breast_cancer_diagnosis_desc')])),
                                              ('classifier',
                                               <catboost.core.CatBoostClassifier object at 0x7ef0d36f8220>)]))],
                 voting='soft')

In [17]:
# Evaluate the ensemble model
y_pred_ensemble = voting_clf.predict_proba(X_valid)[:, 1]
auc_ensemble = roc_auc_score(y_valid, y_pred_ensemble)
print(f'Ensemble (XGBoost + CatBoost): AUC = {auc_ensemble:.4f}')



Ensemble (XGBoost + CatBoost): AUC = 0.8048


In [18]:
# Make predictions on test set
final_predictions_ensemble = voting_clf.predict_proba(women_test)[:, 1]



In [19]:
# Prepare submission file
submission_df = pd.DataFrame({'patient_id': women_test['patient_id'], 'DiagPeriodL90D': final_predictions_ensemble})
submission_df.to_csv('submission.csv', index=False)